In [1]:
import sys
sys.path.append('/home/xinyiz/pamrats')

import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim

# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import average_precision_score
import image.loadImage as loadImage


import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN,MiniBatchKMeans,AgglomerativeClustering
from sklearn import metrics
from sklearn.linear_model import LogisticRegression


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
ifplot=False
ifcount=True

logitL1=0.5 #smaller is sparser
geneThresh=2.5 #times std above mean

inverseAct=None
# inverseAct=None
plottype='umap'
pca=PCA()
minCells=15 #min number of cells for analysis
minCell_clusterDE=5
clustermethod=['leiden']
# clustermethod=['leiden','agglomerative','kmeanbatch']
#umap/leiden clustering parameters
n_neighbors=10
min_dist=0.25
n_pcs=40 #for clustering
resolution=[0.5]
# resolution=[0.05,0.1,0.2,0.3,0.5,0.8,1,1.5]
plotepoch=200
savenameAdd=''
#DBscan
epslist= [6,8,10]
min_sampleslist=[15,30,45] 
#agglomerative
nclusterlist=[2,3,4,5,8,10,15]
aggMetric=['euclidean']


combineCelltype={'glia':['Astro','Micro', 'OPC', 'Oligo'],'CA':['CA1', 'CA2', 'CA3']}

use_cuda=True
fastmode=False #Validate during training pass
seed=3

protein=None #'scaled_binary'
# proteinWeights=0.05
# randFeatureSubset=None
plot_samples={'disease13':'AD_mouse9494','control13':'AD_mouse9498','disease8':'AD_mouse9723','control8':'AD_mouse9735'}
plot_sample_X=['logminmax']
plotRecon='' #'meanRecon'
diamThresh_mul=800
minThresh_mul=12
overlap=int(diamThresh_mul*0.5)
name='all_thresh25min12_03_overlap50'
logsavepath='/mnt/external_ssd/xinyi/log/train_cnn_starmap/'+name
modelsavepath='/mnt/external_ssd/xinyi/models/train_cnn_starmap/'+name
plotsavepath='/mnt/external_ssd/xinyi/plots/train_cnn_starmap/'+name
datadir='/home/xinyiz/2021-01-13-mAD-test-dataset'

In [3]:
# Set cuda and seed
np.random.seed(seed)
if use_cuda and (not torch.cuda.is_available()):
    print('cuda not available')
    use_cuda=False
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.enabled = True


In [4]:
#Load data
imageslist_pi={}
# imageslist_comp={}
imageslist_mask={}
for s in plot_samples.keys():
    imageslist_pi[s]=loadImage.loadandsplit(plot_samples[s],datadir,diamThresh_mul,overlap,0,0,ifFlip=True,minCutoff=minThresh_mul,seed=seed,split=False,imagename='pi_sum.tif',minmaxscale=True)
#     imageslist_comp[s]=loadImage.loadandsplit(plot_samples[s],datadir,diamThresh_mul,overlap,0,0,ifFlip=True,minCutoff=minThresh_mul,seed=seed,split=False,imagename='plaque.tif',minmaxscale=True)
for s in ['disease13','disease8']:
    imageslist_mask[s]=loadImage.loadandsplit(plot_samples[s],datadir,diamThresh_mul,overlap,0,0,ifFlip=True,minCutoff=minThresh_mul,seed=seed,split=False,imagename='Maskofplaque.tif',minmaxscale=True)

(22210, 22344)
flipping images
(22355, 18953)
flipping images
no cells
no cells
no cells
no cells
no cells
no cells
no cells
(22294, 19552)
flipping images
(22452, 19616)
flipping images
(22210, 22344, 4)
flipping images
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no ce

no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
n

no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
n

no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
n

no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
no cells
n

In [5]:
def sampleCluster(labels,imginput,imginput2,nsample,savepath,savenamecluster,addname=''):
    imgsavepath=os.path.join(savepath,savenamecluster+'example')
    if not os.path.exists(imgsavepath):
        os.mkdir(imgsavepath)
    np.random.seed(3)
    for l in np.unique(labels):
        clusteridx=(labels==l)
        idx=np.random.choice(np.sum(clusteridx),min(nsample,np.sum(clusteridx)),False)
        for i in idx:
#             print(np.arange(imginput.shape[0])[clusteridx][idx])
            addname_i=addname[np.arange(imginput.shape[0])[clusteridx][i]]
            imgplot=imginput[np.arange(imginput.shape[0])[clusteridx][i]][0]
            imgplot2=imginput2[np.arange(imginput2.shape[0])[clusteridx][i]][0]
            imgplotcombined=np.stack((imgplot,imgplot2,np.zeros_like(imgplot)),axis=2)
            plt.imsave(os.path.join(imgsavepath,'cluster'+str(l)+'_'+str(i)+addname_i+''+'.jpg'),imgplot)
            plt.imsave(os.path.join(imgsavepath,'cluster'+str(l)+'_'+str(i)+addname_i+'_withPlaque'+'.jpg'),imgplotcombined)            

def countCluster(labels,imginput):
    np.random.seed(3)
    for l in np.unique(labels):
        print('cluster '+str(l))
        clusteridx=(labels==l)
        imgplot=imginput[clusteridx]
        imgcount=np.sum(imgplot,axis=(1,2,3))
        imgcount=np.mean(imgcount)
        print(imgcount)
            
def sampleLeiden(imginput,imginput2,nsample,n_neighbors,n_pcs,min_dist,resolution,addName=''):
    for r in resolution:
        savenamecluster='leiden_nn'+str(n_neighbors)+'mdist0'+str(int(min_dist*100))+'n_pcs'+str(n_pcs)+'res'+str(r)+'epoch'+str(plotepoch)
        readpath=os.path.join(clustersavedir,savenamecluster)
        if not os.path.exists(readpath):
            print('DNE: '+readpath)
            continue
        with open(readpath, 'rb') as input:
            labels = pickle.load(input)
        labels=np.array(labels)
        if np.unique(labels).shape[0]==1:
            continue
        
        savepath=clustersavedir
        if ifcount:
            print(savenamecluster)
            countCluster(labels,imginput)
        if ifplot:
            sampleCluster(labels,imginput,imginput2,nsample,savepath,savenamecluster,addName)

def sampleAgg(imginput,imginput2,nsample,nclusterL,aggmetricL,n_pcs,addName=''):
    for ncluster in nclusterL:
        for aggmetric in aggmetricL:
            savenamecluster='agg_ncluster'+str(ncluster)+aggmetric+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
            readpath=os.path.join(clustersavedir,savenamecluster)
            if not os.path.exists(readpath):
                print('DNE: '+readpath)
                continue
            with open(readpath, 'rb') as input:
                labels = pickle.load(input)
            labels=np.array(labels)
            if np.unique(labels).shape[0]==1:
                continue
            savepath=clustersavedir
            if ifcount:
                print(savenamecluster)
                countCluster(labels,imginput)
            if ifplot:
                sampleCluster(labels,imginput,imginput2,nsample,savepath,savenamecluster,addName)
            
def sampleMinibatchKmean(imginput,imginput2,nsample,nclusterL,n_pcs,addName=''):
    for ncluster in nclusterL:
        savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
        readpath=os.path.join(clustersavedir,savenamecluster)
        if not os.path.exists(readpath):
            print('DNE: '+readpath)
            continue
        with open(readpath, 'rb') as input:
            labels = pickle.load(input)
        labels=np.array(labels)
        if np.unique(labels).shape[0]==1:
            continue
            
        savepath=clustersavedir
        if ifcount:
            print(savenamecluster)
            countCluster(labels,imginput)
        if ifplot:
            sampleCluster(labels,imginput,imginput2,nsample,savepath,savenamecluster,addName)
        
        
def plotCTcomp(labels,ctlist,savepath,savenamecluster,addname=''):
    res=np.zeros((np.unique(labels).size,np.unique(ctlist).size))
    for li in range(res.shape[0]):
        l=np.unique(labels)[li]
        nl=np.sum(labels==l)
        ctlist_l=ctlist[labels==l]
        for ci in range(res.shape[1]):
            c=np.unique(ctlist)[ci]
            res[li,ci]=np.sum(ctlist_l==c)
#             res[li,ci]=np.sum(ctlist_l==c)/nl
    if not byCT:
        addname+=''
        for li in range(res.shape[0]):
            l=np.unique(labels)[li]
            nl=np.sum(labels==l)
            res[li]=res[li]/nl
    else:
        addname+='_normbyCT'
        for ci in range(res.shape[1]):
            c=np.unique(ctlist)[ci]
            nc=np.sum(ctlist==c)
            res[:,ci]=res[:,ci]/nc
    
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(res,cmap='binary')
    ax.set_yticks(np.arange(np.unique(labels).size))
    ax.set_yticklabels(np.unique(labels))
    ax.set_xticks(np.arange(np.unique(ctlist).size))
    ax.set_xticklabels(np.unique(ctlist))
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
    fig.tight_layout()
    plt.savefig(os.path.join(savepath,savenamecluster+'_ctComposition'+addname+'.jpg'))
    plt.close()
        
        

def compLeiden(ctlist,n_neighbors,n_pcs,min_dist,resolution,addName=''):
    for r in resolution:
        savenamecluster='leiden_nn'+str(n_neighbors)+'mdist0'+str(int(min_dist*100))+'n_pcs'+str(n_pcs)+'res'+str(r)+'epoch'+str(plotepoch)
        readpath=os.path.join(clustersavedir,savenamecluster)
        if not os.path.exists(readpath):
            print('DNE: '+readpath)
            continue
        with open(readpath, 'rb') as input:
            labels = pickle.load(input)
        labels=np.array(labels)
        if np.unique(labels).shape[0]==1:
            continue
        
        savepath=clustersavedir
        plotCTcomp(labels,ctlist,savepath,savenamecluster,addName)

def compDBscan(ctlist,epsL,min_samplesL,n_pcs,addName=''):
    for eps in epsL:
        for min_samples in min_samplesL:
            savenamecluster='dbscan_eps'+str(eps)+'msamples'+str(min_samples)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
            readpath=os.path.join(clustersavedir,savenamecluster)
            if not os.path.exists(readpath):
                print('DNE: '+readpath)
                continue
            with open(readpath, 'rb') as input:
                labels = pickle.load(input)
            labels=np.array(labels)
            if np.unique(labels).shape[0]==1:
                continue
            
            savepath=clustersavedir
            plotCTcomp(labels,ctlist,savepath,savenamecluster,addName)
                
def compAgg(ctlist,nclusterL,aggmetricL,n_pcs,addName=''):
    for ncluster in nclusterL:
        for aggmetric in aggmetricL:
            savenamecluster='agg_ncluster'+str(ncluster)+aggmetric+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
            readpath=os.path.join(clustersavedir,savenamecluster)
            if not os.path.exists(readpath):
                print('DNE: '+readpath)
                continue
            with open(readpath, 'rb') as input:
                labels = pickle.load(input)
            labels=np.array(labels)
            if np.unique(labels).shape[0]==1:
                continue
            savepath=clustersavedir
            plotCTcomp(labels,ctlist,savepath,savenamecluster,addName)
            
def compMinibatchKmean(ctlist,nclusterL,n_pcs,addName=''):
    for ncluster in nclusterL:
        savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
        readpath=os.path.join(clustersavedir,savenamecluster)
        if not os.path.exists(readpath):
            print('DNE: '+readpath)
            continue
        with open(readpath, 'rb') as input:
            labels = pickle.load(input)
        labels=np.array(labels)
        if np.unique(labels).shape[0]==1:
            continue
            
        savepath=clustersavedir
        plotCTcomp(labels,ctlist,savepath,savenamecluster,addName)

In [ ]:
#all cells
for s in plot_samples.keys():
    sampleidx=plot_samples[s]
    
    celltype_broad=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']
    celltype_sub=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'cell_type_label']
    region=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']
#     sobj_coord_np=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,['x','y']].to_numpy()
    for xcorr in plot_sample_X:
        samplename=s+'X_'+xcorr
        
        if inverseAct:
            samplename+='_beforeAct'
       
        sampledir=os.path.join(plotsavepath,samplename)
        clustersavedir=os.path.join(plotsavepath,samplename,'cluster')

        if 'leiden' in clustermethod:
            compLeiden(celltype_broad,n_neighbors,n_pcs,min_dist,resolution)
        if 'dbscan' in clustermethod:
            compDBscan(celltype_broad,epslist,min_sampleslist,n_pcs)
        if 'agglomerative' in clustermethod:
            compAgg(celltype_broad,nclusterlist,aggMetric,n_pcs)
        if 'kmeanbatch' in clustermethod:
            compMinibatchKmean(celltype_broad,nclusterlist,n_pcs)

In [13]:
# combine all latents to one plot 
nsample=16

np.random.seed(seed)
latents=None
latents2=None
samplenameList=None

for s in plot_samples.keys():
    sampleidx=plot_samples[s]        
    samplename=s
    muplot,_,_=imageslist_pi[samplename]
    muplot2,_,_=imageslist_comp[samplename]
#     imageslist_pi[samplename]=None
#     imageslist_comp[samplename]=None
    
    if latents is None:
        latents=muplot
        latents2=muplot2
        samplenameList=np.repeat(s,muplot2.shape[0])
    else:
        latents=np.vstack((latents,muplot))
        latents2=np.vstack((latents2,muplot2))
        samplenameList=np.concatenate((samplenameList,np.repeat(s,muplot2.shape[0])),axis=None)
        
sampledir=os.path.join(plotsavepath,'combined')
if inverseAct:
    sampledir+='_beforeAct'
savedir=os.path.join(sampledir,'embedding_'+plottype)
clustersavedir=os.path.join(sampledir,'cluster')

if 'leiden' in clustermethod:
    sampleLeiden(latents,latents2,nsample,n_neighbors,n_pcs,min_dist,resolution,samplenameList)
if 'dbscan' in clustermethod:
    sampleDBscan(latents,latents2,nsample,epslist,min_sampleslist,n_pcs,samplenameList)
if 'agglomerative' in clustermethod:
    sampleAgg(latents,latents2,nsample,nclusterlist,aggMetric,n_pcs,samplenameList)
if 'kmeanbatch' in clustermethod:
    sampleMinibatchKmean(latents,latents2,nsample,nclusterlist,n_pcs,samplenameList)

MemoryError: Unable to allocate 13.3 GiB for an array with shape (2784, 1, 800, 800) and data type float64

In [6]:
# combine all latents to one plot - count
np.random.seed(seed)
latents=None
latents2=None
nsample=None

for s in plot_samples.keys():
    sampleidx=plot_samples[s]        
    samplename=s
    if 'disease' in s:
        muplot,_,_=imageslist_mask[samplename]
    else:
        muplot,_,_=imageslist_pi[samplename]
        muplot=np.zeros_like(muplot)
    
    if latents is None:
        latents=muplot
    else:
        latents=np.vstack((latents,muplot))
        
sampledir=os.path.join(plotsavepath,'combined')
if inverseAct:
    sampledir+='_beforeAct'
savedir=os.path.join(sampledir,'embedding_'+plottype)
clustersavedir=os.path.join(sampledir,'cluster')
        
if 'leiden' in clustermethod:
    sampleLeiden(latents,latents2,nsample,n_neighbors,n_pcs,min_dist,resolution)
if 'dbscan' in clustermethod:
    sampleDBscan(latents,latents2,nsample,epslist,min_sampleslist,n_pcs,samplenameList)
if 'agglomerative' in clustermethod:
    sampleAgg(latents,latents2,nsample,nclusterlist,aggMetric,n_pcs,samplenameList)
if 'kmeanbatch' in clustermethod:
    sampleMinibatchKmean(latents,latents2,nsample,nclusterlist,n_pcs,samplenameList)

MemoryError: Unable to allocate 39.3 GiB for an array with shape (8250, 1, 800, 800) and data type float64

In [ ]:
# combine all latents to one plot - count
np.random.seed(seed)
latents=None
latents2=None
nsample=None

for s in plot_samples.keys():
    sampleidx=plot_samples[s]        
    samplename=s
    muplot,_,_=imageslist_pi[samplename]
    
    if latents is None:
        latents=muplot
    else:
        latents=np.vstack((latents,muplot))
        
sampledir=os.path.join(plotsavepath,'combined')
if inverseAct:
    sampledir+='_beforeAct'
savedir=os.path.join(sampledir,'embedding_'+plottype)
clustersavedir=os.path.join(sampledir,'cluster')
        
if 'leiden' in clustermethod:
    sampleLeiden(latents,latents2,nsample,n_neighbors,n_pcs,min_dist,resolution)
if 'dbscan' in clustermethod:
    sampleDBscan(latents,latents2,nsample,epslist,min_sampleslist,n_pcs,samplenameList)
if 'agglomerative' in clustermethod:
    sampleAgg(latents,latents2,nsample,nclusterlist,aggMetric,n_pcs,samplenameList)
if 'kmeanbatch' in clustermethod:
    sampleMinibatchKmean(latents,latents2,nsample,nclusterlist,n_pcs,samplenameList)

In [ ]:
# combine all latents to one plot -- by samples
np.random.seed(seed)
for xcorr in plot_sample_X:
    samplenameList=None
    celltype_broad=None
    region=None

    for s in plot_samples.keys():
        sampleidx=plot_samples[s]        
        samplename=s+'X_'+xcorr
            
        if samplenameList is None:
            samplenameList=np.repeat(s,np.sum(scaleddata.obs['sample']==sampleidx))
            celltype_broad=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']
            region=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']            
        else:
            celltype_broad=np.concatenate((celltype_broad,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']),axis=None)
            samplenameList=np.concatenate((samplenameList,np.repeat(s,np.sum(scaleddata.obs['sample']==sampleidx))),axis=None)
            region=np.concatenate((region,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']),axis=None)

    origCT=np.unique(celltype_broad)
    celltypeplot=np.concatenate((origCT,list(combineCelltype.keys())),axis=None)
    sampledir=os.path.join(plotsavepath,'combined'+xcorr)
    if inverseAct:
        sampledir+='_beforeAct'
    clustersavedir=os.path.join(sampledir,'cluster')

    if 'leiden' in clustermethod:
        compLeiden(samplenameList,n_neighbors,n_pcs,min_dist,resolution,'Sample')
    if 'dbscan' in clustermethod:
        compDBscan(samplenameList,epslist,min_sampleslist,n_pcs,'Sample')
    if 'agglomerative' in clustermethod:
        compAgg(samplenameList,nclusterlist,aggMetric,n_pcs,'Sample')
    if 'kmeanbatch' in clustermethod:
        compMinibatchKmean(samplenameList,nclusterlist,n_pcs,'Sample')
    #by region
    for reg in np.unique(region):
        savedir=os.path.join(sampledir,'embedding_'+plottype+'_'+reg)
        clustersavedir=os.path.join(sampledir,'cluster'+'_'+reg)

        reg_idx=region==reg

        #by region and celltype
        for ct in celltypeplot:
#             if not ((reg=='Cortex' and ct in ['Ex']) or (reg=='Hippocampus' and ct in ['CA1','DG','Micro','CA'])):
            if not (ct in ['Micro']):
                continue
            print(reg+ct)
            clustersavedir=os.path.join(sampledir,'cluster'+'_'+reg+ct)
            
            if ct in origCT:
                ct_idx=celltype_broad==ct
            else:
                ct_idx=False
                for i in combineCelltype[ct]:
                    ct_idx=np.logical_or(ct_idx,celltype_broad==i)
            ct_idx=np.logical_and(reg_idx,ct_idx)      
            
            if np.sum(ct_idx)<3:
                continue
            
            if 'leiden' in clustermethod:
                compLeiden(celltype_sub[ct_idx],n_neighbors,n_pcs,min_dist,resolution,'Sample')
            if 'dbscan' in clustermethod:
                compDBscan(celltype_sub[ct_idx],epslist,min_sampleslist,n_pcs,'Sample')
            if 'agglomerative' in clustermethod:
                compAgg(celltype_sub[ct_idx],nclusterlist,aggMetric,n_pcs,'Sample')
            if 'kmeanbatch' in clustermethod:
                compMinibatchKmean(celltype_sub[ct_idx],nclusterlist,n_pcs,'Sample')
